In [20]:
!pip install --upgrade \
  azureml-core==1.61.0 \
  azureml-pipeline-core==1.61.0 \
  azureml-pipeline-steps==1.61.0 \
  azureml-train==1.61.0 \
  azureml-train-core==1.61.0 \
  azureml-train-automl==1.61.0 \
  azureml-train-automl-client==1.61.0 \
  azureml-train-automl-runtime==1.61.0 \
  azureml-widgets==1.61.0 \
  azureml-telemetry==1.61.0 \
  azureml-dataset-runtime[fuse,pandas]==1.61.0 \
  azureml-tensorboard==1.61.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.9/420.9 kB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 17.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 47.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 45.9 MB/s eta 0:0

In [1]:
!pip install --upgrade psutil==5.9.8 scipy==1.10.1 pyarrow==14.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 1.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.3
    Uninstalling psutil-5.9.3:
      Successfully uninstalled psutil-5.9.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-nvdashboard 0.13.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
azureml-training-tabular 1.61.0 requires psutil<5.9.4,>=5.2.2, but you have psutil 5.9.8 which is incompatible.
azureml-defaults 1.60.0 requires azureml-core~=1.60.0, but you have azureml-core 1.61.0 which is incompatible.
azureml-defaults 1.60.0 requires azureml-

In [4]:
from azureml.core import Workspace
from azureml.core.compute import AmlCompute, ComputeTarget

# Connect to your workspace using config.json
ws = Workspace.from_config()

# Your compute cluster name
cluster_name = "samirphapale1997"

# Check if cluster exists, else create
if cluster_name in ws.compute_targets:
    compute = ws.compute_targets[cluster_name]
    if type(compute) is AmlCompute:
        print("✅ Found existing cluster:", cluster_name)
else:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_DS11_V2",  # small CPU VM
        min_nodes=0,                 # scale down when idle
        max_nodes=2                  # scale up to 2 nodes
    )
    compute = ComputeTarget.create(ws, cluster_name, compute_config)

# Wait until ready
compute.wait_for_completion(show_output=True)

✅ Found existing cluster: samirphapale1997
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.core import Environment

env = Environment(name="orchestration-env")

# Core Azure ML
env.python.conda_dependencies.add_pip_package("azureml-core")
env.python.conda_dependencies.add_pip_package("azureml-pipeline-core")
env.python.conda_dependencies.add_pip_package("azureml-pipeline-steps")

# Data & ML
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("scikit-learn")

# Additional packages
env.python.conda_dependencies.add_pip_package("pyarrow")
env.python.conda_dependencies.add_pip_package("sqlalchemy")
env.python.conda_dependencies.add_pip_package("pyodbc")
env.python.conda_dependencies.add_pip_package("openpyxl")

# Azure Storage & Identity
env.python.conda_dependencies.add_pip_package("azure-storage-blob")
env.python.conda_dependencies.add_pip_package("azure-identity")

# 🔑 Key Vault SDK
env.python.conda_dependencies.add_pip_package("azure-keyvault-secrets")


# Bind environment to run config
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment = env



In [6]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

# Parameters must be provided by Logic App or SDK run submission
filename_param = PipelineParameter(name="filename",default_value="default.txt")
source_blob_param = PipelineParameter(name="source_blob",default_value="landing_zone/default.txt")

step = PythonScriptStep(
    name="Run Orchestration",
    script_name="orchestration.py",
    compute_target=compute,
    source_directory="/mnt/batch/tasks/shared/LS_root/mounts/clusters/samirphapale19971/code/Users/samirphapale1997",
    arguments=[
        "--filename", filename_param,
        "--source_blob", source_blob_param
    ],
    runconfig=run_config
)

In [7]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(workspace=ws, steps=[step])
published_pipeline = pipeline.publish(name="OrchestrationPipeline1")
print("Pipeline endpoint:", published_pipeline.endpoint)

Created step Run Orchestration [6d48f73c][7b71d866-f4b7-414e-acdc-c44ef0e737ba], (This step will run and generate new outputs)
Pipeline endpoint: https://centralindia.api.azureml.ms/pipelines/v1.0/subscriptions/b291ad3a-8a06-4be6-a22c-e4222ccb2d71/resourceGroups/d-ci-shipping/providers/Microsoft.MachineLearningServices/workspaces/ml-studio/PipelineRuns/PipelineSubmit/fe32f40b-a3cb-4e75-9e3f-77409461d9e0


In [ ]:
# Check keyvault is authorised or not using terminal
# az extension add -n ml --upgrade
# az login
# az ml workspace show -n ml-studio -g d-ci-shipping --query identity
# az keyvault show --name mlstudio1612843333 --query properties.enableRbacAuthorization
# az ml compute update \
#   --name samirphapale1997 \
#   --resource-group d-ci-shipping \
#   --workspace-name ml-studio \
#   --identity-type SystemAssigned

# az role assignment create \
#   --role "Key Vault Secrets User" \
#   --assignee 00ed67d2-aa6f-4a8a-87ec-390bacbbb14d \
#   --scope $(az keyvault show --name mlstudio1612843333 --query id -o tsv)



# save as test.py run on terminal to check 
# from azure.identity import ManagedIdentityCredential
# from azure.keyvault.secrets import SecretClient
# credential = ManagedIdentityCredential()
# client = SecretClient(vault_url="https://mlstudio1612843333.vault.azure.net", credential=credential)
# try:
#     secret = client.get_secret("conn-string")
#     print("Secret value:", secret.value)
# except Exception as e:
#     print("Error:", e)
